## Setup

To run these optional examples, you'll need:

1. The core package and IonQ integration for each SDK example you'd like to run (the installation for these is also included at the beginning of each section)
2. An IonQ API key

Most SDK integrations will automatically find your API key if it's stored as an environment variable named `IONQ_API_KEY` (set external to this notebook, or using the code in the next cell), and most also provide an option to pass in your key directly (so you can import, load, or paste it in if needed).

In [ ]:
import os
os.environ["IONQ_API_KEY"] = "YOUR API KEY HERE"

## Supported SDK examples

While Qiskit is widely used, IonQ also supports a number of other quantum SDKs and frameworks. Here, we'll show a few selected examples; more information is available in our [docs](https://docs.ionq.com) and [resource center](). Not all SDKs currently support all available features and settings, but if there's something you'd like to see from a specific SDK, please let us know!

### Cirq

https://docs.ionq.com/sdks/cirq/index

In [ ]:
%%capture
%pip install cirq cirq-ionq

For Cirq, we use a `Service` to connect to IonQ systems.

In [ ]:
import cirq_ionq
service = cirq_ionq.Service()

# To pass in an API key directly
# service = cirq_ionq.Service(api_key="YOUR API KEY HERE")

Build a circuit:

In [ ]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
qc_cirq = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='x') 
)

print(qc_cirq)

You can use `service.run()` (which waits and returns the result) or `service.create_job()` (async). The backend is specified using the `target` option rather than by creating a separate backend object.

In [ ]:
job_cirq = service.create_job(
    circuit=qc_cirq,
    target="simulator",
    repetitions=1000,
    name="Cirq example",
    extra_query_params={"noise": {"model": "aria-1"}}
)

In [ ]:
print(job_cirq.status())

In [ ]:
job_cirq.job_id()

In [ ]:
result = job_cirq.results()[0].to_cirq_result()
print(result.histogram(key='x'))

### PennyLane

https://docs.ionq.com/sdks/pennylane

In [ ]:
%%capture
%pip install pennylane pennylane-ionq

For PennyLane, setting up an IonQ device handles both the connection and backend selection.

In [ ]:
import pennylane as qml

# Setup the device
dev = qml.device(
    'ionq.simulator',
    #api_key="your_api_key_here",
    wires=2
)

A circuit is defined as a qnode function that targets the specified device and returns results.

In [ ]:
@qml.qnode(dev)
def pennylane_bell_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.probs(wires=[0, 1])

In [ ]:
result = pennylane_bell_state()

In [ ]:
print(result)

### qBraid

https://docs.ionq.com/sdks/qbraid/index

In [ ]:
%%capture
%pip install 'qbraid[ionq]'

Like Qiskit, qBraid's integration uses a provider and a backend (here, a device).

In [ ]:
from qbraid.runtime import IonQProvider
provider = IonQProvider()

# provider = IonQProvider("api key here")

In [ ]:
device = provider.get_device("simulator")

Circuits are defined in qasm, but qBraid's integration converts them to the format accepted by the IonQ API when running. (IonQ has limited support for accepting qasm directly.)

In [ ]:
qasm = """
OPENQASM 3.0;
qubit[2] q;
h q[0];
cx q[0], q[1];
"""

To run the circuit via qBraid, pass it to `device.run()`.

In [ ]:
job = device.run(
    qasm,
    name="qBraid example",
    shots=1000,
    noise={"model" : "aria-1"}
)
result = job.result()

In [ ]:
print(result.data.get_counts())

## CUDA-Q

https://docs.ionq.com/sdks/cuda-q

Local installation of CUDA-Q can be more complicated than other packages depending on your hardware, but this example works in cloud environments like Google Colab.

In [ ]:
%%capture
%pip install cudaq

Specify an IonQ backend and connect to IonQ using `cudaq.set_target`. The target `ionq` is the simulator; QPUs are `ionq_qpu.aria-1`, etc.

Note that CUDA-Q requires the API key to be stored as an environment variable.

In [ ]:
import cudaq
cudaq.set_target('ionq', noise='aria-1')

The circuit is defined as a CUDA-Q kernel:

In [ ]:
@cudaq.kernel
def cudaq_example():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    x.ctrl(qubits[0], qubits[1])

The result is obtained by sampling from the kernel:

In [ ]:
# Run the circuit and print results
result = cudaq.sample(cudaq_example, shots_count=1000)
print(result)

More information about these SDKs and other integrations can be found in our [docs](https://docs.ionq.com/) and [resource center](https://www.ionq.com/resources/).